#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
import os
from kfp import dsl
from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import secretmanager
from google.cloud import language_v1


In [ ]:
DATASET_DISPLAY_NAME = "[your-dataset-display-name]
MODEL_DISPLAY_NAME = '"[your-model-display-name]
ENDPOINT_DISPLAY_NAME = '"[-endpoint-display-name]
#Set the target column representing the suitability or preference score of a stock
TARGET_COLUMN = 'preference_score'
# Set the BigQuery dataset and table names
DATASET_NAME = 'your-bigquery-dataset-name'
TABLE_NAME = 'your-bigquery-table-name'
# Set the Model Registry and Feature Store locations
MODEL_REGISTRY_LOCATION = 'us-central1'
FEATURE_STORE_LOCATION = 'us-central1'
# Set the staging bucket for Vertex AI
STAGING_BUCKET = 'gs://your-staging-bucket'
REGION = 'europe-west3'


In [ ]:
def automl_tabular_train_op(dataset_display_name, target_column, project, location, display_name):
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)
secret_manager_client = secretmanager.SecretManagerServiceClient()
# Custom component for AutoML Tabular training

aiplatform.init(project=project, location=location, staging_bucket=STAGING_BUCKET)
dataset = aiplatform.TabularDataset.get(display_name=dataset_display_name)
model = aiplatform.AutoMLTabularTrainingJob(
display_name=display_name,
optimization_prediction_type='regression',
)
model.run(dataset=dataset, target_column=target_column)
# Save the trained model to the Model Registry
model.save(model_display_name=display_name)


In [ ]:
# Custom component for BigQuery query
def bigquery_query_op(query, project_id, location):
client = bigquery.Client(project=project_id, location=location)
query_job = client.query(query)
query_job.result()


In [ ]:
# Custom component for creating a BigQuery view
def bigquery_create_view_op(query, project_id, location):
client = bigquery.Client(project=project_id, location=location)
query_job = client.query(query)
query_job.result()


In [ ]:
# Custom component for performing sentiment analysis
def sentiment_analysis_op(text):
    # Create a language client
    client = language_v1.LanguageServiceClient()
   # Perform sentiment analysis on the input text
   document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
   response = client.analyze_sentiment(request={'document': document})
   # Extract the sentiment score from the analysis result
   sentiment_score = response.document_sentiment.score
   return sentiment_score


In [ ]:
#Custom component for storing the endpoint ID in Secret Manager
def store_endpoint_id_op(endpoint_id, project_id):
secret_name = 'vertex-ai-endpoint-id'
parent = secret_manager_client.secret_path(project_id, secret_name)
secret = secret_manager_client.create_secret(
request={
"parent": parent,
"secret_id": secret_name,
"secret": {"replication": {"automatic": {}}},
"}
)
secret_payload = endpoint_id.encode("UTF-8")
secret_manager_client.add_secret_version(
request={
"parent": parent,
"payload": {"data": secret_payload},
}
)


In [ ]:
#  Define the pipeline function
@dsl.pipeline(
name='Stock Recommendation Pipeline',
description='Pipeline for training an AutoML model and deploying an endpoint for stock recommendation.'


In [ ]:
def stock_recommendation_pipeline():
# Step 1: Query the BigQuery table to create a view
query = f"""
SELECT user_id, age, gender, income, risk_tolerance, investment_goals, stock_symbol, feedback, sentiment
FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
"""
query_task = bigquery_query_op(
query=query,
project_id=PROJECT_ID,
location=REGION
)



# Step 2: Create the BigQuery view
view_query = f"""
CREATE OR REPLACE VIEW `{PROJECT_ID}.{DATASET_NAME}.narrowed_dataset_view` AS (
SELECT user_id, age, gender, income, risk_tolerance, investment_goals, stock_symbol, feedback, sentiment
FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
WHERE user_id IS NOT NULL AND age IS NOT NULL AND gender IS NOT NULL
)
"""
create_view_task = bigquery_create_view_op(
query=view_query,
project_id=PROJECT_ID,
location=REGION
).after(query_task)


# Step 3: Perform sentimental analysis
sentiment_analysis_task = sentiment_analysis_op(
text=create_view_task.outputs['query'].output_value
).after(create_view_task)


# Step 4: Train an AutoML model
training_task = automl_tabular_train_op(
dataset_display_name=DATASET_DISPLAY_NAME,
target_column=TARGET_COLUMN,
project=PROJECT_ID,
location=REGION,
display_name=MODEL_DISPLAY_NAME
).after(sentiment_analysis_task)


# Step 5: Deploy an endpoint
endpoint_task = aiplatform.EndpointCreateOp(
project=PROJECT_ID,
display_name=ENDPOINT_DISPLAY_NAME
).after(training_task)


# Step 6: Save the model to the Model Registry
save_model_task = aiplatform.ModelUploadOp(
project=PROJECT_ID,
display_name=MODEL_DISPLAY_NAME,
serving_container_image_uri=training_task.outputs['model'].serving_container_image_uri
).after(endpoint_task)


# Step 7: Store the endpoint ID in Secret Manager
store_endpoint_id_task = store_endpoint_id_op(
endpoint_id=endpoint_task.outputs['endpoint'].resource_id,
project_id=PROJECT_ID
).after(save_model_task)


In [ ]:
# Compile the pipeline
pipeline_filename = 'stock_recommendation_pipeline.yaml'
compiler.Compiler().compile(stock_recommendation_pipeline, pipeline_filename)


In [ ]:
# Create an experiment and run the pipeline
experiment = aiplatform.Experiment.create(display_name='Stock Recommendation Experiment')
run = experiment.run(name='Stock Recommendation Run', pipeline=pipeline_filename, enable_caching=False)
